In [1]:
# coding:u8
import json
import numpy as np
import pandas as pd
from tqdm import tqdm as tqdm
import gc
from gensim.models import Word2Vec
from sklearn.model_selection import KFold
from sklearn.externals import joblib

import pickle

/home/zhukaihua/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [ ]:
train_data = []
train_data_dict = {}
with open('data/raw_data/train.json', 'r') as f:
    for line in f:
        raw = eval(str(json.loads(line)).lower())
        train_data.append(raw)
        #train_data_dict[raw['text_id']] = raw
kb = {}
with open('data/raw_data/kb_data', 'r') as f:
    for line in f:
        item = eval(str(json.loads(line)).lower())
        kb[item['subject_id']] = item

name_id = {}
for kb_id in kb:
    for item in kb[kb_id]['alias']:
        if item not in name_id:
            name_id[item] = [kb_id]
        else:
            name_id[item].append(kb_id)
    if kb[kb_id]['subject'] not in name_id:
        name_id[kb[kb_id]['subject']] = [kb_id]
    else:
        name_id[kb[kb_id]['subject']].append(kb_id)
for id in name_id:
    name_id[id] = sorted(list(set(name_id[id])))

In [ ]:
testfile_name = 'data/eval.json'
raw_submit = []
text_idto_text = {}
with open(testfile_name, 'r') as f:
    for line in tqdm(f):
        s = json.loads(line)
        text_idto_text[s['text_id']] = s['text']
        raw_submit.append(s)


In [ ]:
data = pd.read_pickle('data/step1_5_test.pkl')
data2 = pd.read_pickle('data/step2_test.pkl')
data3 = pd.read_pickle('data/step5_test.pkl')

In [ ]:
for c in data2.columns:
    if c not in data.columns:
        data[c] = data2[c]
for c in data3.columns:
    if c not in data.columns:
        data[c] = data3[c]

In [ ]:
predictor=data.columns
exclude = ['text_id',
 'kb_id',
 'train_mention',
 'label',
 'm_id','type']
# exclude = []
temp = []
for x in predictor:
    if x not in exclude :temp.append(x)
predictor2 = temp 

In [ ]:
data

In [7]:
data['text_id']=pd.read_pickle('data/temp.pkl')['text_id']

In [8]:
grades = np.zeros((len(data),4))
for i in range(4):
    gbm_pickle = joblib.load('model_link/lgb_gbdt_%d.pkl'%i)
    grades[:,i]=gbm_pickle.predict(data[predictor2])
    assert len(gbm_pickle.feature_name()) == len(predictor2)

In [7]:
data['text_id'].head()

0    1
1    1
2    1
3    1
4    2
Name: text_id, dtype: object

In [9]:
data['link_grade'] = np.mean(grades,axis=1)

In [10]:
data['link_grade'].head(5)

0    0.991830
1    0.004542
2    0.991998
3    0.003561
4    0.479529
Name: link_grade, dtype: float64

In [13]:
data.shape

(527546, 609)

In [17]:
data.iloc[0:10000,:].describe().loc['mean',:].to_csv('data/mean_feature2.csv',sep='\t')

/home/zhukaihua/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [11]:
# 过滤低预测值数据
valid_data = data[data.link_grade>0.35]
submit = valid_data.groupby('m_id',as_index=False)['link_grade'].agg({'label_id_max':'idxmax'})
m = submit.shape[0]

In [12]:
data['m_id'].nunique()

74998

In [13]:
submit.shape

(71967, 2)

In [14]:
data[['text_id','kb_id','train_mention','label_mean','label_count','m_label_mean',
                                 'm_label_count','link_grade']].head(20)

,text_id,kb_id,train_mention,label_mean,label_count,m_label_mean,m_label_count,link_grade
0,1,340082,中国新歌声,0.967742,31.0,1.00,30.0,0.991830
1,1,24209,快乐男声,0.000000,15.0,0.00,15.0,0.004542
2,1,246979,快乐男声,1.000000,22.0,1.00,15.0,0.991998
3,1,364450,快乐男声,0.000000,15.0,0.00,15.0,0.003561
4,2,171897,林学概论,NaN,NaN,NaN,NaN,0.479529
5,2,57478,林学概论,NaN,NaN,NaN,NaN,0.591996
6,2,20282,简介,0.000000,1488.0,0.00,1488.0,0.000024
7,2,269667,简介,1.000000,1488.0,1.00,1488.0,0.999429
8,2,306472,简介,0.000000,1488.0,0.00,1488.0,0.000010
9,3,396325,小彩旗,1.000000,2.0,1.00,2.0,0.997547


In [16]:
valid_data.shape

(71577, 609)

In [15]:
hit = valid_data.loc[submit['label_id_max'],:]
print(hit.head())
valid_data = hit

   create_works  end_with_line  end_with_line2  end_with_point  \
0             1              0               0               1   
2             0              0               0               1   
5             1              0               1               0   
7             0              0               1               0   
9             0              0               0               0   

   end_with_shuminghao  entity_common   kb_id  label  len_mention    m_id  \
0                    1              1  340082     -1            5  257734   
2                    1              1  246979     -1            4  257735   
5                    1              1   57478     -1            4  257736   
7                    1              1  269667     -1            2  257737   
9                    1              3  396325     -1            3  257738   

   ...  m_rank_mse_distance  mid_cos_distance_var  mid_cos_distance_mean  \
0  ...                  3.0              6.153705               

In [16]:
kbid_finder = {}
for index,row in valid_data.iterrows():
    kbid_finder[str(row['text_id'])+row['train_mention']] = str(row['kb_id'])

In [17]:
# 戴安楠ner
# ner_result = pd.read_csv('result/el_dev_result(1).csv', sep='\t')
# ner_result['offset'] = ner_result['offset'].apply(lambda x: eval(x.lower()))
# ner_result['mention'] = ner_result['mention'].apply(lambda x: eval(x.lower()))
# ner_result = ner_result.rename(columns={'mention': 'pred', 'offset': 'pos'})
# my ner
ner_result = pd.read_pickle('result/ner_bert_result.pkl')
ner_result['text'] = ner_result['text'].apply(lambda x:''.join(x[1:-1]))
ner_result['pos'] = ner_result['pos'].apply(lambda x:[k[0]-1 for k in x])
ner_result['text_id'] = ner_result.index+1
test_data = []
for index, row in ner_result.iterrows():
    item = {}
    item['text_id'] = row['text_id']
    item['text'] = row['text']
    item['mention_data'] = []
    for i, p in enumerate(row['pos']):
        if row['pred'][i] not in name_id:continue
        item['mention_data'].append({'mention': row['pred'][i], 'offset': str(p)})
    test_data.append(item)

In [18]:
kbid_finder

{'1中国新歌声': '340082',
 '1快乐男声': '246979',
 '2林学概论': '57478',
 '2简介': '269667',
 '3小彩旗': '396325',
 '3杨丽萍': '255867',
 '3舞蹈': '153216',
 '4了不起的盖茨比': '315212',
 '4原著': '160942',
 '4世界': '368793',
 '5山寨': '77655',
 '5特效': '151344',
 '5剧情': '196435',
 '6歌曲': '35295',
 '6情醉中华': '217638',
 '6周燕君': '373174',
 '6中国音乐电视': '372982',
 '7技能': '299357',
 '7效果': '313342',
 '7视频': '150546',
 '8漫画': '94209',
 '8卫宫家今天的饭': '304689',
 '8动漫': '135659',
 '9边锋': '278704',
 '9三国杀': '17170',
 '9手游': '236191',
 '10痔疮': '197279',
 '10彻底': '408844',
 '10断根': '321282',
 '10复发': '26617',
 '10大道至简': '399065',
 '11中国': '241286',
 '11人': '368121',
 '11刘赞': '156277',
 '12擂主': '353023',
 '12张艳红': '79062',
 '12李天保娶亲': '394496',
 '13孤光': '195881',
 '13天下3': '337294',
 '13歌曲': '35295',
 '13剧情': '196435',
 '14优酷网': '202883',
 '15旁观霸气侧漏': '266595',
 '15酥油饼': '225042',
 '16沈祖荣': '158033',
 '17看透爱情看透你': '119060',
 '17宋小宝': '24547',
 '17优酷网': '202883',
 '17视频': '150546',
 '18今晚报': '25731',
 '18数字报': '222737',
 '19董事长': '86477',

In [19]:
for item in test_data:
    # if item['kb_id']=='nil':continue
    item['text_id']=str(item['text_id'])
    item['text']=text_idto_text[item['text_id']]
    for m in item['mention_data']:
        try:
            m['kb_id']=kbid_finder[str(item['text_id'])+m['mention']]
        except:
            #print(item,m)
            m['kb_id'] = 'NIL' # 去掉某些ner
        m['mention'] = item['text'][int(m['offset']):int(m['offset'])+len(m['mention'])]
submit_data = test_data

In [20]:
new_submit = []
for item in submit_data:
    new_item = {}
    new_item['text_id'] = item['text_id']
    new_item['text'] = item['text']
    new_item['mention_data'] = []
    for m in item['mention_data']:
        if m['kb_id']=='NIL':continue
        new_item['mention_data'].append(m)
    new_submit.append(new_item)

In [21]:
import json
with open('submit/result_7.json', 'w') as f:
    for line in new_submit:
        json.dump(line,f,ensure_ascii=False)
        f.write('\n')

In [ ]:
data[['text_id','kb_id','train_mention','label_mean','label_count','m_label_mean',
                                 'm_label_count','link_grade']]